In [1]:
import pycaret as pc
import pandas as pd

In [23]:
sell_data = pd.read_excel('./HistoryOfSell_Rev02.xlsx', sheet_name='Sell')

sell_data.head()

,Date,A1,A2,A3,A4,A5,A6,A7,A8,A9,...,A15,A16,A17,A18,A19,A20,A21,A22,A23,A24
0,2014-03-01,2,2,0,1,1,0,2,0,1,...,0,0,1,0,1,0,1,1,0,0
1,2014-04-01,6,4,0,2,4,0,1,1,0,...,0,1,0,2,6,3,5,5,2,2
2,2014-05-01,9,3,0,3,6,0,0,1,0,...,0,0,2,0,4,1,3,1,0,0
3,2014-06-01,8,3,2,6,6,0,2,2,4,...,0,0,2,0,6,2,6,3,0,0
4,2014-07-01,11,4,0,6,4,0,2,3,3,...,0,0,2,0,9,0,6,5,0,0


In [13]:
from summarytools import dfSummary

dfSummary(sell_data)

No,Variable,Stats / Values,Freqs / (% of Valid),Graph,Missing
1,Date[datetime64[ns]],"Min: 2014-03-01Max: 2023-05-01Duration: 3,348 days",111 distinct values,"<img src = ""data:image/png;base64, iVBORw0KGgoAAAANSUhEUgAAAG8AAAA2CAYAAADJRMdfAAAAOXRFWHRTb2Z0d2FyZQBNYXRwbG90bGliIHZlcnNpb24zLjUuMSwgaHR0cHM6Ly9tYXRwbG90bGliLm9yZy/YYfK9AAAACXBIWXMAAAsTAAALEwEAmpwYAAABCElEQVR4nO3VsQkCMRSA4VMsX31X2LiBEziEE2S2TOAU7mFj4dUpD85KEDkxiHL++H9d4BECP+QtxnFsxLSc+wF632ruB/y7tm33EdHVzJZSLn3fH25n480sIrqU0rlmNue8vj/7bYIZD8x4YJM7r3aJllI2EXH61Nw37vz1uWEYtk3TVO28R5PxapdoznmXUjp+au4bdxLmXs0847cJZjww44EZD8x4YMYDMx6Y8cCMB2Y8MOOBGQ/MeGDGAzMemPHAjAdmPDDjgRkPzHhgxgMzHpjxwIwHZjww44EZD8x4YMYDMx6Y8cCMB2Y8MOOBGQ/MeGDGAzMemPHAjAdmPDDjgRkPzHhgV7PYWO+hYbPBAAAAAElFTkSuQmCC"">",0(0.0%)
2,A1[int64],Mean (sd) : 9.7 (5.7)min < med < max:0.0 < 8.0 < 28.0IQR (CV) : 6.0 (1.7),22 distinct values,"<img src = ""data:image/png;base64, iVBORw0KGgoAAAANSUhEUgAAAHoAAAAyCAYAAACTUs/lAAAAOXRFWHRTb2Z0d2FyZQBNYXRwbG90bGliIHZlcnNpb24zLjUuMSwgaHR0cHM6Ly9tYXRwbG90bGliLm9yZy/YYfK9AAAACXBIWXMAAAsTAAALEwEAmpwYAAABUklEQVR4nO3cMWqEQBhA4UkI2Exat0gTvEBygRwiJ5g21/IEOUXukSZFrG0EwRQpl934gxrxva8eUHwwOL/gzTRNScd3+983oG3cbX3Buq5fc86nOWv7vv/uuu597Xsi2Dx0zvlUSvmas7Zt24e174fCrRvC0BCGhjA0hKEhDA2xyPEqcjYex/EppTTreKXlLBI6eDZ+WeKainHrhjA0xOYj0IhhGJ6bpnmbs9a5+HW7Dl1V1b1z8WW4dUMYGsLQEIaGMDSEoSEMDWFoCENDGBrC0BCGhtj1R40Iv3Rdd5jQfum6zq0bwtAQhoYwNIShIQ7z1h1BPIohQxOPYm7dEIaGMDSEoSEMDWFoCENDGBri4sDE31XERZ5ZSin1ff+Yc/5cYe3ZNO9iaH9XERd5Zin9PrdSyscKa8+mecgRaERkLr7nnc3QfwjOxXe7s/kyBvEDwd14SZmlYqUAAAAASUVORK5CYII="">",0(0.0%)
3,A2[int64],Mean (sd) : 5.7 (3.3)min < med < max:0.0 < 5.0 < 23.0IQR (CV) : 4.0 (1.7),16 distinct values,"<img src = ""data:image/png;base64, iVBORw0KGgoAAAANSUhEUgAAAHoAAAAyCAYAAACTUs/lAAAAOXRFWHRTb2Z0d2FyZQBNYXRwbG90bGliIHZlcnNpb24zLjUuMSwgaHR0cHM6Ly9tYXRwbG90bGliLm9yZy/YYfK9AAAACXBIWXMAAAsTAAALEwEAmpwYAAABO0lEQVR4nO3cMWrDMBhAYacUvKirM3QpvkB6gRyiJ9Daa/kEOUXu0aVDPGsxGNwTJEhUcYLf+2YRGx4WRvjPblmWRtv38ugb0Dpea/xI13VfIYR9ztqU0mUcx1ON6ypfldAhhH2M8Tdn7TAM7zWuqTJu3RCGhjA0hKEhDA1haAhDQxgaosqBSYlpmj77vv/OWZtS+ggh/GSu9cTthtVDt237VnCKdowxnjPXeuJ2g1s3hKEhDA1haAhDQxgawtAQhoYwNIShIQwNYWgIQ0MYGsLQEIaGMDTE1S9MSgbn5nk+NE2T9dWIHuNq6MLBuWO9W9I9uHVDGBrC0BCGhjA0hKEhDA1haIjVZ6/upXB4DzeQt5nQhcN7uIG8zYQuQXz6kaGJT78vYxCGhjA0hKEhkC9jJZ7lDf2/f5X9Bx8ZYbkcOqFGAAAAAElFTkSuQmCC"">",0(0.0%)
4,A3[int64],Mean (sd) : 2.6 (2.3)min < med < max:0.0 < 2.0 < 14.0IQR (CV) : 2.5 (1.1),10 distinct values,"<img src = ""data:image/png;base64, iVBORw0KGgoAAAANSUhEUgAAAHoAAAAyCAYAAACTUs/lAAAAOXRFWHRTb2Z0d2FyZQBNYXRwbG90bGliIHZlcnNpb24zLjUuMSwgaHR0cHM6Ly9tYXRwbG90bGliLm9yZy/YYfK9AAAACXBIWXMAAAsTAAALEwEAmpwYAAABHElEQVR4nO3csQ2CQBhAYTQmNGcLhR0TuIoT0LoWEziVhdQ0VDgBhIsnR3jvazWE5IUfuFw4TdNU6PjOuU9A27jM/VBV1SOEUK85yDAMn77vX+lOS6nNhg4h1G3bvtccpOu6W7pT0j84uiEMDWFoCENDGBrC0BCGhjA0hKEhZlfGYozjeG+a5rnmvy6X5pEkdFmWV5dL983RDWFoCENDGBrC0BBJnrpj+CqWx+ahfRXLw9ENYWgIQ0MYGsLQEIaGMDSEoSEMDWFoCENDGBrC0BCGhjA0hKEhDA2x+Q6TGG47SmfXod12lI6jG2LXV3QMx/yyw4R2zC9zdEMYGuIwozsG8X6ODE28nzu6IZBX9L/EfPq6KOJuC79+VvsLeP1r53DWwkkAAAAASUVORK5CYII="">",0(0.0%)
5,A4[int64],Mean (sd) : 3.6 (3.5)min < med < max:0.0 < 3.0 < 22.0IQR (CV) : 4.0 (1.0),14 distinct values,"<img src = ""data:image/png;base64, iVBORw0KGgoAAAANSUhEUgAAAHoAAAAyCAYAAACTUs/lAAAAOXRFWHRTb2Z0d2FyZQBNYXRwbG90bGliIHZlcnNpb24zLjUuMSwgaHR0cHM6Ly9tYXRwbG90bGliLm9yZy/YYfK9AAAACXBIWXMAAAsTAAALEwEAmpwYAAABIklEQVR4nO3cvY3CMBxA8eOElBSmDQVdFoBVmMDtrZUJ6NiIgtRpUoUJgmKd86G892uJIksP/pIti8MwDD/av9+1F6BlHMc+qKrqHkI4T3lJ13Xvtm0f+Zal3EZDhxDOMcbXlJc0TXPJtyTNwdENYWgIQ0MYGsLQEIaGMDSEoSEMDWFoCENDGBrC0BCGhjA0hKEhDA1haAhDQxgawtAQo7dAU/R9f6vr+m/Ks14NXkeW0EVRnLwavG2ObghDQxgawtAQhoYwNESW7VUK99zrWDy0e+51OLohDA1haAhDQxgawtAQhoYwNMTiByYpPEXLZ9OhPUXLx9ENselfdArH/He7Ce2Y/243oeeyl385RoZOGfNlWV5jjM8pz845Kf77hfsAtJpkqYGcv8kAAAAASUVORK5CYII="">",0(0.0%)
6,A5[int64],Mean (sd) : 4.6 (3.3)min < med < max:0.0 < 4.0 < 19.0IQR (CV) : 4.0 (1.4),15 distinct values,"<img src = ""data:image/png;base64, iVBORw0KGgoAAAANSUhEUgAAAHoAAAAyCAYAAACTUs/lAAAAOXRFWHRTb2

In [24]:
import plotly.express as px

for i in range(1, 25):
    sell_data[f'A{i}_Mean'] = sell_data[f'A{i}'].rolling(12).mean()

    fig = px.line(sell_data, x='Date', y=[f'A{i}', f'A{i}_Mean'], template='plotly_dark')
    fig.show()

# A1

In [25]:
import numpy as np

# extract month and year from dates
sell_data['Month'] = [i.month for i in sell_data['Date']]
sell_data['Year'] = [i.year for i in sell_data['Date']]

# create a sequence of numbers
sell_data['Series'] = np.arange(1,len(sell_data)+1)

# drop unnecessary columns and re-arrange

sell_data.drop(['Date', 'A1_Mean'], axis=1, inplace=True)
sell_data = sell_data[['Series', 'Year', 'Month', 'A1']] 

# check the head of the dataset
sell_data.head()

,Series,Year,Month,A1
0,1,2014,3,2
1,2,2014,4,6
2,3,2014,5,9
3,4,2014,6,8
4,5,2014,7,11


In [26]:
# split data into train-test set
train = sell_data[sell_data['Year'] < 2022]
test = sell_data[sell_data['Year'] >= 2022]

# check shape
train.shape, test.shape


((94, 4), (17, 4))

In [27]:
# import the regression module
from pycaret.regression import *

# initialize setup
s = setup(data = train, test_data = test, target = 'A1', fold_strategy = 'timeseries', numeric_features = ['Year', 'Series'], fold = 10, transform_target = True, session_id = 123)

,Description,Value
0,Session id,123
1,Target,A1
2,Target type,Regression
3,Original data shape,"(111, 4)"
4,Transformed data shape,"(111, 4)"
5,Transformed train set shape,"(94, 4)"
6,Transformed test set shape,"(17, 4)"
7,Numeric features,2
8,Preprocess,1
9,Imputation type,simple


In [28]:
best = compare_models(sort = 'MSE')

Processing:   0%|          | 0/77 [00:00<?, ?it/s]

In [29]:
prediction_holdout = predict_model(best);

In [32]:
# generate predictions on the original dataset**
predictions = predict_model(best, data=sell_data)

# add a date column in the dataset**
predictions['Date'] = pd.date_range(start='2014-03-01', end = '2023-05-01', freq = 'MS')

# line plot**
fig = px.line(predictions, x='Date', y=["A1", "prediction_label"], template = 'plotly_dark')

# add a vertical rectange for test-set separation**
fig.add_vrect(x0="2022-07-01", x1="2023-06-01", fillcolor="grey", opacity=0.25, line_width=0)

fig.show()

In [33]:
predictions

,Series,Year,Month,A1,prediction_label,Date
0,1,2014,3,1.256027,2.000000,2014-03-01
1,2,2014,4,2.476401,6.000000,2014-04-01
2,3,2014,5,3.068632,9.000000,2014-05-01
3,4,2014,6,2.888415,8.000000,2014-06-01
4,5,2014,7,3.391394,11.000000,2014-07-01
...,...,...,...,...,...,...
106,107,2023,1,3.537588,9.999255,2023-01-01
107,108,2023,2,5.168150,11.025667,2023-02-01
108,109,2023,3,2.476401,6.643538,2023-03-01
109,110,2023,4,3.537588,5.622903,2023-04-01


In [34]:
final_best = finalize_model(best)

In [35]:
future_dates = pd.date_range(start = '2023-06-01', end = '2024-05-01', freq = 'MS')

future_df = pd.DataFrame()

future_df['Month'] = [i.month for i in future_dates]
future_df['Year'] = [i.year for i in future_dates]    
future_df['Series'] = np.arange(112,(112+len(future_dates)))

future_df.head()

,Month,Year,Series
0,6,2023,112
1,7,2023,113
2,8,2023,114
3,9,2023,115
4,10,2023,116


In [36]:
predictions_future = predict_model(final_best, data=future_df)
predictions_future.head()

,Month,Year,Series,prediction_label
0,6,2023,112,11.203337
1,7,2023,113,11.355895
2,8,2023,114,10.739576
3,9,2023,115,10.918457
4,10,2023,116,13.118724


In [37]:
concat_df = pd.concat([sell_data, predictions_future], axis=0)
concat_df_i = pd.date_range(start='2014-03-01', end = '2024-05-01', freq = 'MS')
concat_df.set_index(concat_df_i, inplace=True)

fig = px.line(concat_df, x=concat_df.index, y=["A1", "prediction_label"], template = 'plotly_dark')
fig.show()